# Mission - Analysez des images médicales avec des méthodes semi-supervisées

Vous êtes Data Scientist junior spécialisé en Computer Vision au sein de CurelyticsIA, une startup innovante dans le domaine de la e-santé. L’entreprise développe des solutions basées sur l’intelligence artificielle pour assister les professionnels de santé dans l’analyse d’images médicales, en particulier des IRM.
 
Dans le cadre d’un nouveau projet R&D, CurelyticsIA souhaite explorer la possibilité d’automatiser la détection de tumeurs du cerveau. Un ensemble conséquent de radios a été collecté : la majorité de ces images ne dispose d’aucun étiquetage, tandis qu’un sous-ensemble limité a été annoté par des radiologues experts.
 
Vous êtes chargé de concevoir une première exploration analytique du jeu de données. Plus précisément, votre mission est de :
- Explorer les images et extraire des caractéristiques visuelles via un modèle pré-entraîné ;
- Appliquer des méthodes de clustering pour identifier des structures ou regroupements dans les données ;
- Mettre en œuvre une méthode d’apprentissage semi-supervisé à partir des quelques étiquettes disponibles ;
- Synthétiser vos résultats, formuler des recommandations, et les présenter à votre équipe projet.

**Mail à prendre en compte :**

Comme discuté lors de notre dernière réunion, tu es assigné à la première phase du projet BrainScanAI. Tu trouveras en pièce jointe un fichier zip contenant :
- Le jeu de données de radiographies (en format PNG + métadonnées anonymisées),
- Une documentation technique sur le format des images ;
- Une liste restreinte de labels annotés par nos partenaires hospitaliers (normal/cancéreux). 

Pour info, notre budget actuel pour la labellisation par IA est de 300 euros pour ce dataset. 

Tes objectifs :
1) Extraire des caractéristiques visuelles pertinentes à l’aide d’un modèle pré-entraîné (type ResNet ou équivalent).
2) Réaliser un clustering exploratoire pour identifier des regroupements naturels.
3) Mettre en œuvre une méthode semi-supervisée en exploitant les labels partiels pour prédire les étiquettes manquantes.
4) Proposer des livrables au format Notebook contenant :
    - l’extraction des features
    - le preprocessing adapté au(x) modèle(s) utilisés
    - l’analyse non-supervisée (.ipynb)
    - l’entraînement de modèles de clustering
    - l’approche semi-supervisée (.ipynb)
 
Ces livrables doivent être accompagnés d’un support de présentation proposant des recommandations techniques pour un passage à l’échelle (budget de 5 000 euros pour 4 millions d’images à labelliser). Est-ce que ce passage te paraît faisable et si oui, sous quelles conditions ?

5) Rédiger une synthèse de ton approche et de tes résultats dans un support de présentation. Les contraintes :
    - Travailler en Python.
    - Tester plusieurs algorithmes.
    - Avoir des métriques pertinentes en fonction de l’erreur la plus importante (F1, Acc, Précision, ou autre ?).
    - Clairement définir ce que tu considères comme un objectif atteint (“definition of done”).


# Étape 2 - Prétraitez et extrayez les features
Préparez les images (redimensionnement, normalisation) et utilisez un modèle pré-entraîné (ex : ResNet) pour extraire des embeddings visuels.
 
**Prérequis**
- Avoir nettoyé et formaté les données image.
- Avoir compris le fonctionnement des CNNs.

**Résultat attendu** 
- Vecteurs de features pour chaque image, sauvegardés dans un tableau exploitable.

**Recommandations**
- Geler les couches convolutionnelles.
- Évaluer plusieurs couches d’extraction si besoin.

**Outils**
- Torchvision
- TensorFlow
- Transforms
- Numpy 
- Pandas 
- Matplotlib
- Opencv-python


## Importation des librairies

In [1]:
# Librairies de base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Librairies spécifiques
import os # permet de travailler avec le système de fichiers
from PIL import Image # ouvrir et manipuler des images

# Librairies PyTorch
import torchvision
import torch
from torchvision import transforms # pour effectuer les changements de format, entre autres
from torchvision.models import resnet18, ResNet18_Weights # chargement du modèle ResNet 18
import torch.nn as nn

### Pour extraire les embeddings de nos images, nous devons faire quelques transformations sur celles-ci afin de respecter la documentation de ResNet (modèle pré-entaîné utilisé ici). Nous avons suivi cette dernière pour transformer nos images médicales :
- https://pytorch.org/hub/pytorch_vision_resnet/

#### Preprocess afin :
* de réduire la taille
* de les transformer en tensor
* de les normaliser 
* pour respecter les attentes du ResNet

In [2]:
preprocess = transforms.Compose([
    transforms.Resize(256), # car entraînement ImageNet sur la taille 256
    transforms.CenterCrop(224), # ResNet a été entraîné sur du 224x224
    transforms.ToTensor(), # transformation de l'image PIL en tensor PyTorch
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalisation du RGB selon les stats ResNet
])

#### Choix du modèle pré-entrâiné - ResNet18
* Les filtres des convolutions, les biais, etc. sont déjà appris.
* Le modèle sait déjà extraire des caractéristiques visuelles utiles (bords, textures, formes, objets).
* On utilise **model.fc = nn.Identity()** afin d'utiliser l'avant dernière couche pour ne pas générer la classification mais avoir uniquement les embeddings par image.

In [3]:
model = resnet18(weights=ResNet18_Weights.DEFAULT) # 18 fait référence au nombre de couches
model.fc = nn.Identity()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### Fonction pour appliquer le preprocess/le model et l'enregistrement des vecteurs dans une liste à l'ensemble des dossiers

In [9]:
def extract_embeddings(folder_path):
    """ Génération de l'ouverture d'une image d'un dossier à l'extraction de ses features """
    
    embeddings = [] # enregistrement des éléments dans une liste
    total = 0

    for img_name in os.listdir(folder_path): # chemin du dossier
        img_path = os.path.join(folder_path, img_name) # chemin pour récupérer les images
        img = Image.open(img_path).convert("RGB") # on ouvre une image après l'autre et on force en RGB au cas où

        
        input_tensor = preprocess(img) # on applique le preprocess pour que l'image corresponde aux attentes de ResNet
        input_batch = input_tensor.unsqueeze(0) # besoin d'un espace en 4 dimensions pour le modèle ResNet

        with torch.no_grad(): # désactive le calcul du gradient
            emb = model(input_batch) # lancement du modèle
            emb = emb.squeeze(0)
        embeddings.append(emb.numpy()) # enregistrement de chaque transformation avec les 512 caractéristiques
        total +=1
        
    print(f"Nombre total d’images : {total}")
    print(f"Shape : {embeddings[0].shape}")

    return embeddings
    

#### Application sur le dossier normal

In [10]:
embeddings_normal = extract_embeddings("../mri_dataset_brain_cancer_oc/avec_labels/normal")


Nombre total d’images : 50
Shape : (512,)


* On retrouve nos 50 images mais avec nos 512 caractéristiques

#### Application sur le dossier cancer

In [11]:
embeddings_cancer = extract_embeddings("../mri_dataset_brain_cancer_oc/avec_labels/cancer")

Nombre total d’images : 50
Shape : (512,)


#### Application sur le dossier sans label

In [12]:
embeddings_sans_label = extract_embeddings("../mri_dataset_brain_cancer_oc/sans_label")

Nombre total d’images : 1406
Shape : (512,)


### Enregistrement en DataFrame, on constitue nos jeux de données

In [32]:
df_normal = pd.DataFrame(np.asarray(embeddings_normal))
df_cancer = pd.DataFrame(np.asarray(embeddings_cancer))
df_unlabeled = pd.DataFrame(np.asarray(embeddings_sans_label))

#### On identifie les labels : 0 pour normal et 1 pour cancer

In [41]:
df_normal["label"] = 0
df_cancer["label"] = 1

#### On rassemble nos 2 jeux labellisés

In [38]:
df_labeled = pd.concat([df_normal, df_cancer], ignore_index=True)

### Sauvegarde des DataFrame

In [42]:
df_labeled.to_csv("../data/processed/df_labeled.csv", index=False)
df_unlabeled.to_csv("../data/processed/df_unlabeled.csv", index=False)